#Applying Data Modelling to the One Big table #

###WE always start with the Dimensional Table creating 1st at last we create Fact Table ###

In [0]:
%sql
select * from datamodeling.silver.silver_table

###Find the theme in one big table ###

####**Customer_id, customer_name, customer_email these are the dim table**

In [0]:
%sql
select customer_id, customer_name, customer_email, Customer_Name_Upper from datamodeling.silver.silver_table

###DeDuplication need to perform in Dim Table###

In [0]:
%sql
select distinct(customer_id), customer_name, customer_email, Customer_Name_Upper from datamodeling.silver.silver_table

###Always 1st remove duplicate and then create surrogate key becuase the surrogate key is based on the unique key###

###Creating Surrogate Key for dim table to join with fact table###

In [0]:
%sql
select row_number() over(order by customer_id) as DimCustomerKey, customer_name, customer_email, Customer_Name_Upper from datamodeling.silver.silver_table

In [0]:
%sql
--Method:1 Using SubQuery
/*select *, row_number() over(order by customer_id) as DimCustomerKey from(
  select distinct(customer_id),
  customer_name, customer_email, Customer_Name_Upper 
from datamodeling.silver.silver_table)*/

--Method:2 Using CTE
with rem_dup as (
  select distinct(customer_id),
  customer_name, customer_email, Customer_Name_Upper 
from datamodeling.silver.silver_table)
select *, row_number() over(order by customer_id) as DimCustomerKey from rem_dup

###creating table for Dim Customer###

In [0]:
%sql
create or replace table datamodeling.gold.DimCustomers as
with rem_dup as (
  select distinct(customer_id),
  customer_name, customer_email, Customer_Name_Upper 
from datamodeling.silver.silver_table)
select *, row_number() over(order by customer_id) as DimCustomerKey from rem_dup

In [0]:
%sql
select * from datamodeling.gold.DimCustomers

###Creating Product Dim ###

In [0]:
%sql
select distinct(product_id),product_name,product_category from datamodeling.silver.silver_table
    


###Creating Surr_key for product Dim Table ###

In [0]:
%sql
create or replace table datamodeling.gold.DimProducts as
with rem_dup as (
  select distinct(product_id),
  product_name, product_category 
from datamodeling.silver.silver_table)
select *, row_number() over(order by product_id) as DimProductKey from rem_dup

In [0]:
%sql
select * from datamodeling.gold.DimProducts

###Creating DimPayements###

In [0]:
%sql
select payment_type from datamodeling.silver.silver_table

###DeDuplication for payment and creating surrogate key and creating dim payment table###

In [0]:
%sql
create or replace table datamodeling.gold.DimPayments as
with rem_dup as (
  select distinct(payment_type)
from datamodeling.silver.silver_table)
select *, row_number() over(order by payment_type) as DimPaymentKey from rem_dup

    
    

    


In [0]:
%sql
select * from datamodeling.gold.DimPayments

###Creating DimRegion###

In [0]:
%sql
create or replace table datamodeling.gold.DimRegions as
with rem_dup as (
  select distinct(country)
from datamodeling.silver.silver_table)
select *, row_number() over(order by country) as DimRegionKey from rem_dup

In [0]:
%sql
select * from datamodeling.gold.dimregions

###Creating theme for original table except quanity and unit price columns excluded###

###Creating DimSales###

####Select column names from the source table ####

In [0]:
spark.sql("select * from datamodeling.silver.silver_table").columns

In [0]:
%sql
create or replace table datamodeling.gold.DimSales as
with rem_dup as (
select
distinct(order_id),
 order_date,
 customer_id,
 customer_name,
 customer_email,
 product_id,
 product_name,
 product_category,
 payment_type,
 country,
 last_updated,
 Customer_Name_Upper,
 processDate
from datamodeling.silver.silver_table)
select *, row_number() over(order by order_id) as DimSaleKey from rem_dup

In [0]:
%sql
select * from datamodeling.gold.dimsales

###Creating FACT Table from the One Big Table ###

####To Create Fact Table we need to take Numberic Column like Quanity & Unit_price from 1 Big Table and all the DimKeys####

###Joining them with each and every Dim Table using Surrogate Key###

In [0]:
%sql
select c.DimCustomerKey,p.DimProductKey,r.DimRegionKey,py.DimPaymentKey,s.DimSaleKey, F.quantity,F.unit_price 
from datamodeling.silver.silver_table F
left join datamodeling.gold.dimcustomers c on F.customer_id = c.customer_id
left join datamodeling.gold.DimProducts p on F.product_id = p.product_id
left join datamodeling.gold.DimRegions r on F.country = r.country
left join datamodeling.gold.DimPayments py on F.payment_type = py.payment_type 
left join datamodeling.gold.DimSales s on F.order_id = s.order_id


##Creating Fact_Table ##

In [0]:
%sql
create or replace table datamodeling.gold.FactSales as
select c.DimCustomerKey,p.DimProductKey,r.DimRegionKey,py.DimPaymentKey,s.DimSaleKey, F.quantity,F.unit_price 
from datamodeling.silver.silver_table F
left join datamodeling.gold.dimcustomers c on F.customer_id = c.customer_id
left join datamodeling.gold.DimProducts p on F.product_id = p.product_id
left join datamodeling.gold.DimRegions r on F.country = r.country
left join datamodeling.gold.DimPayments py on F.payment_type = py.payment_type 
left join datamodeling.gold.DimSales s on F.order_id = s.order_id

In [0]:
%sql
select * from datamodeling.gold.FactSales